In [232]:
import numpy as np
import pandas as pd
import trimesh as tm
from matplotlib import pyplot as plt
import torch as tc
from torch import Tensor, cdist

np.set_printoptions(precision=6, suppress=True)

In [270]:
def load_template(file_path):
        faces = []
        normals = []
        i = 0
        with open(file_path) as file:
            for line in file:
                if line[0] == "f":
                    line_split = line[1:].split()
                    line_split = np.array([i.split("//") for i in line_split])
                    face_temp = list(map(int, line_split[:, 0]))
                    faces.append(face_temp)
                    i += 1
        return np.array(faces), np.array(normals)

def load_mesh(file_path):
    vertex = []
    with open(file_path, "r") as f:
        for line in f:
            if line[0] == '#':
                continue
            elif "v " in line:
                line.replace('\n', ' ')
                tmp = list(map(float, line[1:].split()))
                vertex.append(tmp)
            else:
                break
    vertex = np.array(vertex)
    return vertex

def save_obj(path, pontos, faces=[]):
    with open(path, "w") as file:
        for ponto in pontos:
            file.write("v {} {} {}\n".format(ponto[0], ponto[1], ponto[2]))
        for face in faces:
            file.write("f {} {} {}\n".format(face[0], face[1], face[2]))

def generate_positions(vertices, coordinates):
    x = vertices[coordinates[:,0].to(tc.long)].T * coordinates[:,3]
    y = vertices[coordinates[:,1].to(tc.long)].T * coordinates[:,4]
    z = vertices[coordinates[:,2].to(tc.long)].T * coordinates[:,5]
    p = x+y+z
    return p.T

vertices = load_mesh('../curves-modules/mean_female_pose_0.obj')
faces, _ = load_template('../curves-modules/mean_female_pose_0.obj')
faces -= 1

In [283]:
def baricentric(a, b, c, p):
    
    vab = b - a
    vbc = c - b
    vca = a - c
    
    vap = p - a
    vbp = p - b
    vcp = p - c


    cross_norm = tc.linalg.norm(tc.cross(vab,vbc), axis=1)
    n = tc.cross(vab,vbc) / tc.column_stack([cross_norm, cross_norm, cross_norm])
    
    ABC = (n * tc.cross(vab, vbc)).sum(axis=1) / 2
    ABP = (n * tc.cross(vab, vbp)).sum(axis=1) / 2
    BCP = (n * tc.cross(vbc, vcp)).sum(axis=1) / 2
    CAP = (n * tc.cross(vca, vap)).sum(axis=1) / 2

    w = ABP/ABC
    u = BCP/ABC
    v = CAP/ABC

    colisions = tc.column_stack([u,v,w])
    return colisions

def intersection(p0, p1, p2, f1,f2,f3):

    vd = p0 - p1
    integer_part = tc.inner(p0, nplane)
    incognita_part = tc.inner(nplane, vd)
    t = -(dplane + integer_part)/incognita_part
    p = p0 + (tc.column_stack([t,t,t])*vd)
    return tc.column_stack([tc.column_stack([f1,f2,f3]), baricentric(p0,p1,p2,p)])


def calculate_colision(vertices, faces):
    global test
    v = vertices[faces]
    v = v.reshape(-1, 3)
    colision = (v@nplane) + dplane
    colision = colision.reshape(-1, 3)
    v = v.reshape(-1,3,3)

    t1 = tc.all(colision < 0, axis=1)
    t2 = tc.all(colision > 0, axis=1)
    t3 = ~(tc.any(tc.column_stack((t1,t2)), axis=1))

    f = faces[t3]
    colision = colision[t3]
    v = v[t3]
    
    t1 = colision[:,1] * colision[:,2] > 0
    t2 = colision[:,0] * colision[:,2] > 0
    t3 = colision[:,0] * colision[:,1] > 0

    p = tc.row_stack([
        intersection(v[t1][:,0], v[t1][:,1], v[t1][:,2], f[t1][:,0], f[t1][:,1], f[t1][:,2]),
        intersection(v[t1][:,0], v[t1][:,2], v[t1][:,1], f[t1][:,0], f[t1][:,2], f[t1][:,1]),

        intersection(v[t2][:,1], v[t2][:,2], v[t2][:,0], f[t2][:,1], f[t2][:,2], f[t2][:,0]),
        intersection(v[t2][:,1], v[t2][:,0], v[t2][:,2], f[t2][:,1], f[t2][:,0], f[t2][:,2]),

        intersection(v[t3][:,2], v[t3][:,0], v[t3][:,1], f[t3][:,2], f[t3][:,0], f[t3][:,1]),
        intersection(v[t3][:,2], v[t3][:,1], v[t3][:,0], f[t3][:,2], f[t3][:,1], f[t3][:,0])
    ])

    return p

def sort_curve(positions, measure):
    centroid = positions.mean(axis=0)
    vectors = positions - centroid
    sort_around = vectors.var(axis=0).argmin()
    if sort_around == 0:
        angles = tc.arctan2(vectors[:,1], vectors[:,2])
    if sort_around == 1:
        angles = tc.arctan2(vectors[:,0], vectors[:,2])
    if sort_around == 2:
        angles = tc.arctan2(vectors[:,0], vectors[:,1])
    return measure[tc.argsort(angles)]

min_x = vertices[:,0].min()
max_x = vertices[:,0].max()

coordinates_array = []
test = []
for i in np.arange(min_x, max_x, 1e-3):
    nplane, dplane = tc.Tensor([1.0,0.0,0.0]), -i
    coordinates = calculate_colision(tc.Tensor(vertices), tc.from_numpy(faces))
    if coordinates.size:
        positions = generate_positions(tc.Tensor(vertices), coordinates)
        coordinates = sort_curve(positions, coordinates)
        coordinates_array.append(coordinates)
np.save('x.npy', np.array(coordinates_array))


/tmp/ipykernel_205185/934517236.py:94: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  np.save('x.npy', np.array(coordinates_array))
/tmp/ipykernel_205185/934517236.py:94: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.save('x.npy', np.array(coordinates_array))


In [282]:
def calculate_distances(positions):
    positions = Tensor(positions)
    distances = cdist(positions, positions)
    distance = distances[0].sum() + distances[:,0][-1]
    return distance

measures_y = []
for curve in np.load('y.npy', allow_pickle=True):
    positions = generate_positions(tc.Tensor(vertices), tc.Tensor(curve))
    girth = calculate_distances(positions)
    measures_y.append(girth*100)

measures_x = []
for curve in np.load('x.npy', allow_pickle=True):
    positions = generate_positions(tc.Tensor(vertices), tc.Tensor(curve))
    girth = calculate_distances(positions)
    measures_x.append(girth*100)

In [299]:
life_data = pd.read_csv('../curves-modules/datasets/life.csv').dropna()
life_data[life_data['Gender']=='female']
life_data = life_data.drop(['Unnamed: 0', 'Body Type', 'Record Number', 'Age', 'Gender', 'BMI',
        'WHR', 'WHtR', 'ABSI', 'Age'], axis=1)
life_data

,Height,Weight,HEAD_HT,NECK_HT,DIST_NECK_T_HIP,DIST_NECK_KNEE,DIST_WAIST_KNEE,DIST_WAISTBAND_KNEE,WAISTBAND_HT,WAIST_HT,...,DIST_WAISTBAND_HIP,WAIST_T_BUTTOCK,DIST_WAISTBAND_BUTT,NECK_DEPTH,BREAST_DEPTH,WAISTBAND_DEPTH,BELLY_DEPTH,MAX_BELLY_DEPTH,HIP_DEPTH,BUTTOCK_DEPTH
0,170.6,58.5,24.4,146.2,59.3,99.8,61.5,54.2,100.2,107.8,...,0.5,21.0,13.6,9.3,23.2,18.6,18.5,18.5,22.4,21.9
1,175.6,60.7,22.9,152.7,62.7,104.7,63.0,56.2,102.3,110.9,...,0.7,21.5,14.1,9.3,20.5,19.1,19.2,19.4,22.9,23.0
2,181.2,67.8,23.5,157.7,64.6,108.0,65.9,58.5,106.4,115.5,...,1.3,23.5,15.0,10.0,24.5,19.5,19.5,19.7,23.4,23.3
3,180.1,77.8,24.0,156.1,63.2,106.6,64.7,60.6,109.8,114.2,...,3.6,21.5,17.3,11.0,26.9,24.5,24.8,25.0,24.5,25.9
4,177.7,67.2,25.4,152.3,61.5,103.9,63.4,54.1,101.5,111.8,...,1.8,21.4,11.7,9.9,23.0,20.2,20.2,20.3,22.9,24.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8398,177.8,106.2,21.1,156.7,57.5,103.8,62.7,57.2,105.0,115.6,...,2.7,16.6,10.9,15.3,35.1,32.1,38.4,38.5,37.0,36.2
8399,168.2,83.8,20.4,147.8,56.0,98.9,60.0,53.7,97.5,108.9,...,2.2,16.9,10.9,13.3,30.1,29.6,33.7,33.7,32.4,31.5
8400,173.9,78.5,21.1,152.8,58.5,102.5,61.6,51.2,99.0,111.9,...,6.2,17.5,7.1,13.4,31.3,24.7,31.2,31.1,25.6,25.7
8401,181.2,97.3,22.6,158.6,63.4,107.9,65.1,56.2,102.4,115.8,...,1.9,20.8,11.8,13.2,31.4,30.3,36.0,35.7,33.8,32.3


In [318]:
measures_y = tc.Tensor(measures_y).to(tc.float64)
data = tc.from_numpy(life_data.to_numpy()).to(tc.float64)

In [367]:
result = (data.T @ measures_y.repeat(data.shape[0],1))
#ta errado